In [ ]:
#|default_exp ffcv.operations

In [ ]:
#|exporti
# Contains code from:
# FFCV - Apache License 2.0 - Copyright (c) 2022 FFCV Team

# FFCV Operations
> Operations for the fastxtend `Loader`

fastxtend provides multiple FFCV operations, including existing FFCV operations as [a reference](#ffcv-operation-reference), a fastai compatible `ToDevice`, and fastai compatible [Tensor conversions](#convert-to-fastai-tensors).

By default, these operations are imported under `ft` if using `from fastxtend.ffcv.all import *`.

In [ ]:
#|export
from __future__ import annotations

from abc import ABCMeta
from typing import Callable, Optional, Tuple
from dataclasses import replace

import torch
import numpy as np

from fastcore.dispatch import retain_meta
from fastcore.transform import _TfmMeta

from fastai.data.transforms import IntToFloatTensor as _IntToFloatTensor

from ffcv.pipeline.allocation_query import AllocationQuery
from ffcv.pipeline.operation import Operation
from ffcv.pipeline.state import State
from ffcv.transforms.ops import ToDevice as _ToDevice
from ffcv.transforms.ops import Convert, View

from fastxtend.imports import *

In [ ]:
#|export
_all_ = ['Convert', 'View']

In [ ]:
#|hide
from nbdev.showdoc import show_doc

## FFCV Operations Reference

These operations are from FFCV. You can find the original documentation at the [FFCV API Reference](https://docs.ffcv.io/api/transforms.html).

### Convert -

In [ ]:
show_doc(Convert)

### Convert -

In [ ]:
show_doc(View)

## ToDevice

While FFCV has a [<code>ToDevice</code>](https://docs.ffcv.io/api/transforms.html#ffcv.transforms.ToTensor) operation, it is recommended to use the fastxtend `ToDevice` operation for compatability with fastai features.

In [ ]:
#|export
class ToDevice(_ToDevice):
    "Move tensor to device and retains metadata"
    def __init__(self,
        device:int|str|torch.device='cuda', # Device to move Tensor to
        non_blocking:bool=True # Asynchronous if copying from CPU to GPU
    ):
        device, *_ = torch._C._nn._parse_to(device=device)
        super().__init__(device, non_blocking)

    def generate_code(self) -> Callable:
        def to_device(inp, dst):
            if len(inp.shape) == 4:
                if inp.is_contiguous(memory_format=torch.channels_last):
                    dst = dst.reshape(inp.shape[0], inp.shape[2], inp.shape[3], inp.shape[1])
                    dst = dst.permute(0,3,1,2)
            if not isinstance(dst, type(inp)):
                dst = retain_meta(dst, torch.as_subclass(dst, type(inp)))
            dst = dst[:inp.shape[0]]
            dst.copy_(inp, non_blocking=self.non_blocking)
            return dst
        return to_device

## Convert to fastai Tensors

While FFCV has [`ToTensor`](https://docs.ffcv.io/api/transforms.html#ffcv.transforms.ToTensor) and [`ToTorchImage`](https://docs.ffcv.io/api/transforms.html#ffcv.transforms.ToTorchImage) operations for converting NumPy arrays to PyTorch Tensors, it is recommended to use these fastxtend operations for compatability with fastai features.

### ToTensorBase -

In [ ]:
#|export
class ToTensorBase(Operation):
    "Convert from Numpy array to fastai TensorBase or `tensor_cls`."
    def __init__(self, tensor_cls:TensorBase=TensorBase):
        super().__init__()
        self.tensor_cls = tensor_cls

    def generate_code(self) -> Callable:
        tensor_cls = self.tensor_cls
        def to_tensor(inp, dst):
            return tensor_cls(torch.from_numpy(inp))
        return to_tensor

    def declare_state_and_memory(self, previous_state: State) -> Tuple[State, Optional[AllocationQuery]]:
        new_dtype = torch.from_numpy(np.empty((), dtype=previous_state.dtype)).dtype
        return replace(previous_state, jit_mode=False, dtype=new_dtype), None

### ToTensorImage -

In [ ]:
#|export
class ToTensorImage(ToTensorBase):
    "Convenience operation to convert from Numpy array to fastai TensorImage or `tensor_cls`."
    def __init__(self, tensor_cls:TensorImageBase=TensorImage):
        super().__init__()
        self.tensor_cls = tensor_cls

    def generate_code(self) -> Callable:
        tensor_cls = self.tensor_cls
        def to_tensor(inp, dst):
            return tensor_cls(torch.from_numpy(inp).permute(0,3,1,2))
        return to_tensor

### ToTensorImageBW -

In [ ]:
#|export
class ToTensorImageBW(ToTensorImage):
    "Convenience operation to convert from Numpy array to fastai TensorImageBW."
    def __init__(self):
        super().__init__(TensorImageBW)

### ToTensorMask -

In [ ]:
#|export
class ToTensorMask(ToTensorImage):
    "Convenience operation to convert from Numpy array to fastai TensorMask."
    def __init__(self):
        super().__init__(TensorMask)

### ToTensorCategory -

In [ ]:
#|export
class ToTensorCategory(ToTensorBase):
    "Convenience operation to convert from Numpy array to fastxtend TensorCategory."
    def __init__(self):
        super().__init__(TensorCategory)

### ToTensorMultiCategory -

In [ ]:
#|export
class ToTensorMultiCategory(ToTensorBase):
    "Convenience operation convert from Numpy array to fastxtend TensorMultiCategory."
    def __init__(self):
        super().__init__(TensorMultiCategory)

### ToTensorTitledTensorScalar -

In [ ]:
#|export
class ToTensorTitledTensorScalar(ToTensorBase):
    "Convenience operation convert from Numpy array to fastai TitledTensorScalar."
    def __init__(self):
        super().__init__(TitledTensorScalar)